In [1]:
# Dependencies:
# pip: scikit-learn, anndata, scanpy
#
# Modified from the Python starter kit for the NeurIPS 2021 Single-Cell Competition.
# Parts with `TODO` are supposed to be changed by you.
#
# More documentation:
#
# https://viash.io/docs/creating_components/python/

In [2]:
import logging
import anndata as ad
import sys

from scipy.sparse import csc_matrix

from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LinearRegression
import numpy as np

logging.basicConfig(level=logging.INFO)

In [3]:
## VIASH START
# Anything within this block will be removed by `viash` and will be
# replaced with the parameters as specified in your config.vsh.yaml.
meta = { 'resources_dir': '.' }

par = {
    'input_train_mod1': 'sample_data/openproblems_bmmc_multiome_starter/openproblems_bmmc_multiome_starter.train_mod1.h5ad',
    'input_train_mod2': 'sample_data/openproblems_bmmc_multiome_starter/openproblems_bmmc_multiome_starter.train_mod2.h5ad',
    'input_test_mod1': 'sample_data/openproblems_bmmc_multiome_starter/openproblems_bmmc_multiome_starter.test_mod1.h5ad',
    'distance_method': 'minkowski',
    'output': 'output.h5ad',
    'n_pcs': 50,
}
## VIASH END

In [4]:
method_id = 'basic_beans'
sys.path.append(meta['resources_dir'])

In [5]:
logging.info('Reading `h5ad` files...')
input_train_mod1 = ad.read_h5ad(par['input_train_mod1'])
input_train_mod2 = ad.read_h5ad(par['input_train_mod2'])
input_test_mod1 = ad.read_h5ad(par['input_test_mod1'])

input_train = ad.concat(
    {"train": input_train_mod1, "test": input_test_mod1},
    axis=0,
    join="outer",
    label="group",
    fill_value=0,
    index_unique="-"
)

INFO:root:Reading `h5ad` files...


In [6]:

# TODO: implement own method
from method import method
adata = method(input_train_mod1, input_train_mod2, input_test_mod1)

adata.uns["method_id"] = method_id



INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:root:TO DO: Calculate something useful...
INFO:root:Calculate mean of the training data modality 2...


In [7]:

from scipy.sparse import issparse
issparse(adata.X)

True

In [ ]:
issparse(x2)

In [ ]:

logging.info('Storing annotated data...')
adata.write_h5ad(par['output'], compression = "gzip")
